### Install dependencies

In [ ]:
!pip install opensmile

  Created wheel for iso-639: filename=iso_639-0.4.5-py3-none-any.whl size=169062 sha256=d3daaa453329b3ce50108c2fe13652930dcb16a4b36e468fef2b86164cac257a
  Stored in directory: c:\users\j\appdata\local\pip\cache\wheels\ed\ce\cc\1961a4de7090b2e92895fb087abfa0080a542a5706c5948bcc
Successfully built iso-639
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 5.3.1


ERROR: Cannot uninstall 'PyYAML'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


### File preprocessing functions

In [ ]:
import numpy as np 
import pandas as pd 
# import opensmile
import IPython
import os

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler


In [ ]:
# Load data

def preprocess():

  csv_path = 'C:/Users/junxi/Desktop/ADReSSo21/diagnosis/train/adresso-train-mmse-scores.csv'
  df = pd.read_csv(csv_path)

  df['path'] = '/content/drive/MyDrive/ADReSSo21/diagnosis/train/audio/' + df['dx'].astype(str) + '/' + df['adressfname'].astype(str) + '.wav'

  # Replacing AD = 1 (Have alzheimer's) 
  # Replacing CN = 0 (No alzheimer's)
  df['dx'] = df['dx'].replace(['ad'], 1)
  df['dx'] = df['dx'].replace(['cn'], 0)

  df = df[['path', 'dx']]
  
  return df


### Functions to extract features

In [ ]:
# Function to extract features

def feature_extraction(df):

  # Loading opensmile and using eGeMAPS feature set
  smile = opensmile.Smile(
      feature_set=opensmile.FeatureSet.eGeMAPSv02,
      feature_level=opensmile.FeatureLevel.Functionals,
  )

  features_df = pd.DataFrame(columns=smile.feature_names)

  # Extract features for each audio file
  for i in range(len(df)):
    audio_path = df['path'][i]
    temp_features = smile.process_file(audio_path)
    features_df = features_df.append(temp_features.iloc[0], ignore_index=True)

    # Append filename and labels
    features_df.at[i, 'file'] = audio_path
    features_df.at[i, 'label'] = df['dx'].loc[df['path'] == audio_path].values[0]

    print(f'Row {i} done.' )

  return features_df


In [ ]:
def get_features_df():

  # Get dataframe with path and labels
  df = preprocess()

  # Extract features 
  features_df = feature_extraction(df)

  # Convert labels to integer for AD classification
  features_df['label'] = features_df['label'].astype(int)

  return features_df

### Extracting features

In [ ]:
# Extract features
features_df = get_features_df()

# Save features csv file
features_df.to_csv('/content/features_df.csv', index=False)

### Load features and data preprocessing

In [ ]:
# Since we saved the features to a csv file, we can load it

def split_data(features_df):

  features_df = features_df.drop('file', 1)

  # Split to features and labels
  Y = features_df['label']
  X = features_df.drop('label', 1)

  # Normalize data
  scaler = StandardScaler()
  X2 = scaler.fit_transform(X)

  return X2, Y

# Load features
features_df = pd.read_csv('C:/Users/j/Desktop/ADReSSo21/Audio_Classification/features_df.csv')

# Split data into features / labels
X_train, y_train = split_data(features_df)

C:\Users\j\AppData\Local\Temp\ipykernel_11928\3983766178.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  features_df = features_df.drop('file', 1)
C:\Users\j\AppData\Local\Temp\ipykernel_11928\3983766178.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = features_df.drop('label', 1)


### Machine Learning Classifiers

#### Import libraries

In [ ]:
# Sklearn for machine learning classifiers
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.svm import SVC 
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

# For multi processing
import joblib
import sys
sys.modules['sklearn.externals.joblib'] = joblib
from sklearn.externals.joblib import parallel_backend

#### Decision Tree

Functions to perform grid search cross validation for each classifiers


In [ ]:
def decision_tree(X_train, y_train):
  
  param_grid = {
  'max_depth': np.arange(1,11),
  }

  tree = DecisionTreeClassifier()

  # Grid Search for 10 fold cv
  grid = GridSearchCV(tree, param_grid, refit=True, n_jobs=2, cv=10)
  with parallel_backend('threading'):

    grid.fit(X_train, y_train)
    MS = (grid.cv_results_['mean_test_score'][grid.best_index_])
    STD = (grid.cv_results_['std_test_score'][grid.best_index_])
    print('Mean Test Score:', MS)
    print('Std Test Score:', STD) 

#### Decision Tree Bagger

In [ ]:
def decision_tree_bagger(X_train, y_train):
  param_grid = {
      'base_estimator__max_depth' : np.arange(1,11),
  }

  tree = DecisionTreeClassifier()

  # Grid Search
  grid = GridSearchCV(BaggingClassifier(tree), param_grid, refit=True, n_jobs=2, cv=10)
  with parallel_backend('threading'):

    grid.fit(X_train, y_train)
    MS = (grid.cv_results_['mean_test_score'][grid.best_index_])
    STD = (grid.cv_results_['std_test_score'][grid.best_index_])
    print('Mean Test Score:', MS)
    print('Std Test Score:', STD) 

#### Random Forest

In [ ]:
def random_forest(X_train, y_train):
  param_grid = { 
      'n_estimators': [50],
      'max_depth' : np.arange(1,11),
    }

  rf = RandomForestClassifier()

  # Grid Search
  grid = GridSearchCV(rf, param_grid, refit=True, n_jobs=2, cv=10)
  with parallel_backend('threading'):

    grid.fit(X_train, y_train)
    MS = (grid.cv_results_['mean_test_score'][grid.best_index_])
    STD = (grid.cv_results_['std_test_score'][grid.best_index_])
    print('Mean Test Score:', MS)
    print('Std Test Score:', STD) 

#### Random Forest Bagger

In [ ]:
def random_forest_bagger(X_train, y_train):
  param_grid = {
      'base_estimator__max_depth' : np.arange(1,11),
      'base_estimator__n_estimators' : [50] }

  rf = RandomForestClassifier()

  # Grid Search
  grid = GridSearchCV(BaggingClassifier(rf), param_grid, refit=True, n_jobs=2, cv=10)
  with parallel_backend('threading'):

    grid.fit(X_train, y_train)
    MS = (grid.cv_results_['mean_test_score'][grid.best_index_])
    STD = (grid.cv_results_['std_test_score'][grid.best_index_])
    print('Mean Test Score:', MS)
    print('Std Test Score:', STD) 

#### Support Vector 

In [ ]:
def support_vector(X_train, y_train):
  param_grid = {
      'C': np.arange(0.5, 1.6, 0.1),  
      'kernel': ['linear', 'rbf']
      }  

  svc = SVC(max_iter=30000)  

  # Grid Search
  grid = GridSearchCV(svc, param_grid, refit=True, n_jobs=2, cv=10)
  with parallel_backend('threading'):

      grid.fit(X_train, y_train)
      MS = (grid.cv_results_['mean_test_score'][grid.best_index_])
      STD = (grid.cv_results_['std_test_score'][grid.best_index_])
      print('Mean Test Score:', MS)
      print('Std Test Score:', STD) 

#### Logistic Regression

In [ ]:
def log_reg(X_train, y_train):
  param_grid = {
      'C': np.arange(0.5, 1.6, 0.1),  
      }  

  logreg = LogisticRegression(max_iter=10000)
  
  # Grid Search
  grid = GridSearchCV(logreg, param_grid, refit=True, n_jobs=2, cv=10)
  with parallel_backend('threading'):

      grid.fit(X_train, y_train)
      print('Best Params:', grid.best_params_)
      MS = (grid.cv_results_['mean_test_score'][grid.best_index_])
      STD = (grid.cv_results_['std_test_score'][grid.best_index_])
      print('Mean Test Score:', MS)
      print('Std Test Score:', STD) 

#### Training 10 fold cross validation

In [ ]:
# Run all functions

print('Decision Tree')
decision_tree(X_train, y_train)

print('Decision Tree Bagger')
decision_tree_bagger(X_train, y_train)

print('Random Forest')
random_forest(X_train, y_train)

print('Random Forest Bagger')
random_forest_bagger(X_train, y_train)

print('SVM')
support_vector(X_train, y_train)

print('Logistic Regression')
log_reg(X_train, y_train)


Decision Tree
Mean Test Score: 0.6323529411764707
Std Test Score: 0.10727485072753966
Decision Tree Bagger
Mean Test Score: 0.68125
Std Test Score: 0.08990530509421034
Random Forest
Mean Test Score: 0.6871323529411765
Std Test Score: 0.08291969501752466
Random Forest Bagger
Mean Test Score: 0.6566176470588235
Std Test Score: 0.1347535556534434
SVM
Mean Test Score: 0.7110294117647059
Std Test Score: 0.08779300915646644
Logistic Regression
Best Params: {'C': 0.9999999999999999}
Mean Test Score: 0.7058823529411764
Std Test Score: 0.09756164318754212


### Neural Network

#### Import libraries

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import classification_report, confusion_matrix 

from keras.layers import Dense, Dropout, LeakyReLU
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam

import tensorflow as tf
tf.random.set_seed(42)
np.random.seed(42)


#### Function to create model

In [ ]:
# Define neural network model

def create_model(neuron, leaky_alpha, dropout_rate, learning_rate):
    
  print ('Creating model...')

  model = Sequential()

  # Total of 88 features from eGeMAPS
  model.add(Dense(neuron, input_dim=88))
  model.add(LeakyReLU(alpha=leaky_alpha))
  model.add(Dropout(dropout_rate))

  model.add(Dense(neuron))
  model.add(LeakyReLU(alpha=leaky_alpha))
  model.add(Dropout(dropout_rate))

  model.add(Dense(neuron))
  model.add(LeakyReLU(alpha=leaky_alpha))

  model.add(Dense(1, activation='sigmoid'))

  print ('Compiling...')
  optimizer = Adam(learning_rate=learning_rate)

  # For binary classification AD or non AD
  model.compile(loss='binary_crossentropy',
                optimizer=optimizer,
                metrics=['accuracy'])
  
  return model

#### Training and tuning model

In [ ]:
# 10 fold cross validation
cvkf = KFold(n_splits=10, shuffle=True)

# Tune hyper-parameters
params = {
'batch_size':[1,2,4], 
'epochs':[50],
'leaky_alpha':[0.2],
'dropout_rate':[0.3,0.5],
'learning_rate':[1e-3],
'neuron':[16, 32],
}

model = KerasClassifier(build_fn=create_model)

# Grid Search
grid = GridSearchCV(model, 
                param_grid=params,
                n_jobs=2,
                cv=cvkf,
                refit=True)  

with parallel_backend('threading'):
  grid = grid.fit(X_train, y_train)

print('Mean Test Score:', grid.cv_results_['mean_test_score'][grid.best_index_])
print('Std Test Score:', grid.cv_results_['std_test_score'][grid.best_index_])

# Return the best model
best_model = grid.best_estimator_

C:\Users\j\AppData\Local\Temp\ipykernel_11928\3519633491.py:20: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model)


Creating model...
Creating model...
Compiling...
Compiling...
Epoch 1/50
Epoch 1/50
38/38 [==============================] - 1s 9ms/step - loss: 0.9651 - accuracy: 0.4564
Epoch 2/50
38/38 [==============================] - 1s 9ms/step - loss: 0.9533 - accuracy: 0.5034
Epoch 2/50
38/38 [==============================] - 0s 9ms/step - loss: 0.7779 - accuracy: 0.5772
Epoch 3/50
38/38 [==============================] - 0s 10ms/step - loss: 0.9085 - accuracy: 0.5034
Epoch 3/50
38/38 [==============================] - 0s 8ms/step - loss: 0.7371 - accuracy: 0.6107
Epoch 4/50
38/38 [==============================] - 0s 8ms/step - loss: 0.8644 - accuracy: 0.5235
Epoch 4/50
38/38 [==============================] - 0s 8ms/step - loss: 0.9305 - accuracy: 0.4966
Epoch 5/50
38/38 [==============================] - 0s 8ms/step - loss: 0.7771 - accuracy: 0.5101
Epoch 5/50
38/38 [==============================] - 0s 8ms/step - loss: 0.7056 - accuracy: 0.5839
Epoch 6/50
38/38 [==========================

In [ ]:
print(grid.best_params_)

In [ ]:
# Train on best parameters

# Best parameters
# params = {
# 'batch_size':[4], 
# 'epochs':[50],
# 'leaky_alpha':[0.2],
# 'dropout_rate':[0.5],
# 'learning_rate':[1e-3],
# 'neuron':[32],
# }

final_model = create_model(32, 0.2, 0.5, 1e-3)
final_model.fit(X_train, y_train, epochs=50, batch_size=4, validation_split=0.2, shuffle=True)

Creating model...
Compiling...
Epoch 1/50
33/33 [==============================] - 1s 9ms/step - loss: 0.7657 - accuracy: 0.6364 - val_loss: 1.2352 - val_accuracy: 0.0294
Epoch 2/50
33/33 [==============================] - 0s 5ms/step - loss: 0.8276 - accuracy: 0.6288 - val_loss: 1.1059 - val_accuracy: 0.0882
Epoch 3/50
33/33 [==============================] - 0s 5ms/step - loss: 0.7580 - accuracy: 0.6212 - val_loss: 1.0767 - val_accuracy: 0.1176
Epoch 4/50
33/33 [==============================] - 0s 5ms/step - loss: 0.6930 - accuracy: 0.6288 - val_loss: 1.0682 - val_accuracy: 0.1471
Epoch 5/50
33/33 [==============================] - 0s 5ms/step - loss: 0.6803 - accuracy: 0.6136 - val_loss: 1.0858 - val_accuracy: 0.1471
Epoch 6/50
33/33 [==============================] - 0s 5ms/step - loss: 0.6633 - accuracy: 0.6515 - val_loss: 1.0808 - val_accuracy: 0.1765
Epoch 7/50
33/33 [==============================] - 0s 5ms/step - loss: 0.7046 - accuracy: 0.6364 - val_loss: 1.0741 - val_accura

### Save audio model

In [ ]:
# Save the final model

final_model.save('C:/Users/j/Desktop/ADReSSo21/audio_model2')

INFO:tensorflow:Assets written to: C:/Users/j/Desktop/ADReSSo21/audio_model2\assets
